In [5]:
# Import dependencies
import pandas as pd
import hvplot.pandas
import geopandas as gpd
from shapely.geometry import Point

In [3]:
# Read in the CSV file as a Pandas DataFrame
IWSS_df = pd.read_csv(
    Path("Resources/IWSS clustering.csv")
)
site_no_copy = IWSS_df["Site No."].copy()
IWSS_df = IWSS_df.drop("Site No.", axis=1)
IWSS_df['Lat'] = pd.to_numeric(IWSS_df['Lat'], errors='coerce')
IWSS_df['Long'] = pd.to_numeric(IWSS_df['Long'], errors='coerce')
# Review the DataFrame
IWSS_df.head()

,Lat,Long
0,-31.861,116.278
1,-31.900,115.974
2,-31.837,116.038
3,-31.914,116.133
4,-31.867,116.221


In [4]:
IWSS_df.hvplot.scatter(x="Lat", y="Long")

:Scatter   [Lat]   (Long)

In [14]:
map_plot_1 = IWSS_df.hvplot.points("Long", "Lat", xlabel='Longitude', ylabel='Latitude', geo=True, tiles="OSM", size=50, width=800, height=500)
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Long,Lat]

In [15]:
# Start by importing the K-means algorithm
from sklearn.cluster import KMeans

In [16]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [17]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(IWSS_df)
    inertia.append(k_model.inertia_)

C:\Users\mh30f\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\mh30f\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\mh30f\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\mh30f\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are

In [18]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,10.378869
1,2,3.978331
2,3,2.622178
3,4,1.407050
4,5,1.120652


In [19]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

## Using k-means with four clusters

In [20]:
# Create and initialise the K-means model instance for 2 clusters
model = KMeans(n_clusters=4, random_state=1)

# Print the model
model

KMeans(n_clusters=4, random_state=1)

In [21]:
# Fit the data to the instance of the model
model.fit(IWSS_df)

C:\Users\mh30f\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\mh30f\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [22]:
# Make predictions about the data clusters using the trained model
IWSStripgroup = model.predict(IWSS_df)

# Print the predictions
print(IWSStripgroup)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 2
 2 2 2 2 2 2 0 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 1 1 3 1 1 1 1 1 1 0 1
 0 1 1 1 1 3 3 1 1 1 3 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [23]:
# Create a copy of the DataFrame
IWSS_predictions_df = IWSS_df.copy()

# Add a column to the DataFrame that contains the customer_ratings information
IWSS_predictions_df['IWSS_trip_group'] = IWSStripgroup
IWSS_predictions_df['Lat'] = pd.to_numeric(IWSS_predictions_df['Lat'], errors='coerce')
IWSS_predictions_df['Long'] = pd.to_numeric(IWSS_predictions_df['Long'], errors='coerce')

# Review the DataFrame
IWSS_predictions_df.head()

,Lat,Long,IWSS_trip_group
0,-31.861,116.278,0
1,-31.900,115.974,0
2,-31.837,116.038,0
3,-31.914,116.133,0
4,-31.867,116.221,0


In [24]:
# Plot the data points based on the customer rating
IWSS_predictions_df.hvplot.scatter(
    x="Lat", 
    y="Long", 
    by="IWSS_trip_group"
)

:NdOverlay   [IWSS_trip_group]
   :Scatter   [Lat]   (Long)

In [33]:
IWSS_predictions_df['Site No.'] = site_no_copy
IWSS_predictions_df.head()

,Lat,Long,IWSS_trip_group,Site No.
0,-31.861,116.278,0,6556W003
1,-31.900,115.974,0,6055W032
2,-31.837,116.038,0,6056W033
3,-31.914,116.133,0,6072W001
4,-31.867,116.221,0,6082W002


In [26]:
IWSS_predictions_df.to_csv('IWSS_predictions.csv', index=False)

In [37]:
map_plot_2 = IWSS_predictions_df.hvplot.points("Long","Lat", xlabel='Longitude', ylabel='Latitude', geo = True,tiles = "OSM",size =50,
                                        color = "IWSS_trip_group",hover_cols=["Site No."],
                                        tools=["hover"],width=1000,height=800)
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Long,Lat]   (IWSS_trip_group,Site No.)